In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Preprocessing the Data for a Neural Network

In [2]:
# Attempt #1: Remove noisy variables in addition to 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME', 'ASK_AMT', 'STATUS'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,N,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,N,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,N,0
34297,T5,Independent,C3000,ProductDev,Association,0,N,1


### Bin Application Type

In [3]:
# Determine which values to replace if counts are less than ...?
replace_app_types = list(application_df.APPLICATION_TYPE.value_counts()[application_df.APPLICATION_TYPE.value_counts() < 500].index)

# Replace in dataframe
for app in replace_app_types:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

### Bin Classification Type

In [4]:
# Determine which values to replace if counts are less than ..?
replace_class = list(application_df.CLASSIFICATION.value_counts()[application_df.CLASSIFICATION.value_counts() < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,N,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,N,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,N,0
34297,T5,Independent,C3000,ProductDev,Association,0,N,1


In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Design a neural network model, taking into account any modifications that will optimize the model to achieve higher than 75% accuracy.

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
# Attempt #2: Add neurons to HL1
hidden_nodes_layer1 =  190 # not 80

#Attempt #3 (failed): Added 40 neurons to HL2, but it decreased the accuracy score, so I put it back at 30.
hidden_nodes_layer2 = 30

#Attempt #4: Added hidden layer (failed)
# hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
# Attempt #5 (failed) Change output activation style
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid")) # not relu or tanh

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 190)               7980      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                5730      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 13,741
Trainable params: 13,741
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/optimized_weights.{epoch:02d}.hdf5"

In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 750us/step - loss: 0.5828 - accuracy: 0.7086
Epoch 2/100
804/804 [==============================] - 1s 748us/step - loss: 0.5593 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 1s 740us/step - loss: 0.5516 - accuracy: 0.7323
Epoch 4/100
804/804 [==============================] - 1s 772us/step - loss: 0.5508 - accuracy: 0.7298
Epoch 5/100
779/804 [============================>.] - ETA: 0s - loss: 0.5433 - accuracy: 0.7377
Epoch 00005: saving model to checkpoints/optimized_weights.05.hdf5
804/804 [==============================] - 1s 791us/step - loss: 0.5435 - accuracy: 0.7376
Epoch 6/100
804/804 [==============================] - 1s 789us/step - loss: 0.5467 - accuracy: 0.7352
Epoch 7/100
804/804 [==============================] - 1s 812us/step - loss: 0.5478 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 1s 803us/step - loss: 0.5485 - accuracy: 0.7322
Epoch 9/100
804/804 [===

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5596 - accuracy: 0.7250
Loss: 0.559633731842041, Accuracy: 0.7250145673751831


In [16]:
orig_model_acc = .7257142663002014
if model_accuracy < orig_model_acc:
    print(f"This attempt was less successful than the original")
else:
    print(f"This attempt was more successful than the original")

This attempt was less successful than the original


In [17]:
# Export our model to HDF5 file
nn.save("Exports/optimal_trained_attrition.h5")

I can not figure out why the "optimized accuracy" is less successful than the original accuracy, when I only left in variable changes that improved the accuracy. However, when I ran all the code with the saving features, suddenly the accuracy changed. Why??